# Assignment 3 ( Question 4) Know the hatred!

## Instructions
- Run this notebook on ```Google Colab(preferable)```
- Write your code and analysis in the indicated cells.
- Ensure that this notebook runs without errors when the cells are run in sequence.
- Do not attempt to change the contents of other cells. 

## Packages Used
- all the packages that are imported in the template code

## Submission
- Rename the notebook to `<roll_number>_Assignment3_Q4.ipynb`.


In [1]:
pip install scikit-multilearn

     |████████████████████████████████| 89 kB 2.7 MB/s 


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import re
import sklearn
import torch
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier

In [4]:
!pip install transformers
import transformers
from transformers import AutoTokenizer, AutoModel

     |████████████████████████████████| 4.0 MB 5.1 MB/s 
     |████████████████████████████████| 596 kB 45.4 MB/s 
     |████████████████████████████████| 6.6 MB 36.1 MB/s 
     |████████████████████████████████| 895 kB 44.0 MB/s 
     |████████████████████████████████| 77 kB 6.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


#Dataset

In the training data, the comments are labelled as one or more of the six categories; toxic, severe toxic, obscene, threat, insult and identity hate. This is essentially a multi-label classification problem.

The dataset here is a multi-label classification dataset. To understand multi-label classification datasets, you can refer here: https://www.analyticsvidhya.com/blog/2017/08/introduction-to-multi-label-classification/ 

.


Here, we will be using Binary relevance to solve our multi-label classification problem



**Binary Relevance**: This is probably the simplest which treats each label as a separate single classification problems. The key assumption here though, is that there are no correlation among the various labels.

Finally, we will be summing up the accuracies obtained of all labels to obtain the final accuracy


In [5]:
# loading the dataset

train_df = pd.read_csv('/content/q4_data/train.csv')
test_df = pd.read_csv('/content/q4_data/test.csv')

# train_df = pd.read_csv('../input/question4/train.csv')
# test_df = pd.read_csv('../input/question4/test.csv')

#dataset understanding
train_df.sample(5)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
24598,40ff83492b21c363,I can find no evidence \n\nthat Evelyn Beatr...,0,0,0,0,0,0
82723,dd3ef00751a8710b,"""Welcome to Wikipedia, the free encyclopedia! ...",0,0,0,0,0,0
71329,bef18777d85b9873,Origins of the Kingdom of Alba\nI feel that th...,0,0,0,0,0,0
158138,e94272539b29dd70,hi guys\n\ni see youve redirected the clairse...,0,0,0,0,0,0
39366,690bc9da86f010f5,I've read that section several times earlier t...,0,0,0,0,0,0


In [6]:
# let's see the total rows in train, test data and the numbers for the various categories
print('Total rows in test is {}'.format(len(test_df)))
print('Total rows in train is {}'.format(len(train_df)))
# print(train_df[cols_target].sum())


Total rows in test is 153164
Total rows in train is 159571


In [7]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text


In [8]:
train_df['comment_text'] = train_df['comment_text'].map(lambda com : clean_text(com))
test_df['comment_text'] = test_df['comment_text'].map(lambda com : clean_text(com))

In [9]:
# train_df = train_df.drop('char_length',axis=1)
X = train_df.comment_text
test_X = test_df.comment_text
print(X.shape, test_X.shape)


(159571,) (153164,)


Now, we will be experimenting this classification problem with different classifiers. We will be using pre-trained models inorder to obtain the required embeddings. Here, we will be using BERT and Universal Sentence Encoder embeddings.


Understand how Elmo and USE(Universal Sentence Encoder) works here. You can use any of both to train your classifiers.

To understand how different sentence embeddings work, you can refer to this link: https://neptune.ai/blog/how-to-code-bert-using-pytorch-tutorial 

Please give it a look before you start working with transfer learning/ sentence embeddings like BERT, Elmo, Ulm-fit, USE.


In [10]:
#loading universal sentence encoder

import tensorflow_hub as hub
# Load pre-trained universal sentence encoder model
use_embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
# elmo_embed = hub.Module("https://tfhub.dev/google/elmo/3", trainable=True)

In [11]:
def get_use_embedding(sentence):
  embedding = use_embed([sentence])
  embedding = embedding.numpy()
  return embedding[0]

In [12]:
#to obtain bert embeddings ( you need not completely understand how it works)
#you can directly use the function get_bert_embedding to obtain the embedding


#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')
model = AutoModel.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')


def get_bert_embedding(sentence):
  encoded_input = tokenizer(sentence, padding=True, truncation=True, return_tensors='pt')
  with torch.no_grad():
    model_output = model(**encoded_input)
  sentence_embedding = mean_pooling(model_output, encoded_input['attention_mask'])
  return sentence_embedding[0]


Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

In [13]:
labels = pd.read_csv('/content/q4_data/test_labels.csv')
labels = labels[labels.toxic!=-1]
test = pd.merge(test_df, labels)
test

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0001ea8717f6de06,thank you for understanding i think very highl...,0,0,0,0,0,0
1,000247e83dcc1211,dear god this site is horrible,0,0,0,0,0,0
2,0002f87b16116a7f,somebody will invariably try to add religion r...,0,0,0,0,0,0
3,0003e1cccfd5a40a,it says it right there that it is a type the t...,0,0,0,0,0,0
4,00059ace3e3e9a53,before adding a new product to the list make s...,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
63973,fff8f64043129fa2,jerome i see you never got around to this i m ...,0,0,0,0,0,0
63974,fff9d70fe0722906,lucky bastard http wikimediafoundation org wik...,0,0,0,0,0,0
63975,fffa8a11c4378854,shame on you all you want to speak about gays ...,0,0,0,0,0,0
63976,fffac2a094c8e0e2,mel gibson is a nazi bitch who makes shitty mo...,1,0,1,0,1,0


In [14]:
%%time 
#Obtain X_test and X_train
#update 10000 to len(X) later
X_train_use = []
for i in range(30000):
  X_train_use.append(np.array(get_use_embedding(X[i])))
X_train_use = np.array(X_train_use)

CPU times: user 5min 58s, sys: 1min 8s, total: 7min 6s
Wall time: 4min 24s


In [15]:
X_test = test.comment_text

In [16]:
%%time
#update 10000 to len(test)
X_test_use = []
for i in range(10000):
    X_test_use.append(np.array(get_use_embedding(X_test[i])))
X_test_use = np.array(X_test_use)


CPU times: user 1min 58s, sys: 22.5 s, total: 2min 20s
Wall time: 1min 26s


In [17]:
X_train_use = np.array(X_train_use)
X_train_use.shape

(30000, 512)

In [18]:
y_train = train_df[['toxic','severe_toxic','obscene', 'threat','insult', 'identity_hate']]
y_train = y_train[0:30000]

In [19]:
y_test = test[['toxic','severe_toxic','obscene', 'threat','insult', 'identity_hate']]
y_test = y_test[0:10000]
y_test

,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0
...,...,...,...,...,...,...
9995,0,0,0,0,0,0
9996,0,0,0,0,0,0
9997,0,0,0,0,0,0
9998,0,0,0,0,0,0


#### Due to limited Computational resources, the given dataset has been divided as follows : 

- Embedding used - Universal Sentence Encoder(USE)
- Train set - 30000 points
- Test set - 10000 points

# Classifiers
Here, we use different classifiers for our classification problem. You can directly import classifiers for first two.


*   Naive Bayes classifier
*   Support Vector Machine
*   Multi-layer Perceptron(MLP)

In [20]:
# Naive Bayes classifier 
classifierNB = BinaryRelevance(GaussianNB())
classifierNB.fit(X_train_use, y_train)
y_pred_nb = classifierNB.predict(X_test_use)
print("Accuracy on Naive Bayes classifier : ", accuracy_score(y_test, y_pred_nb))

Accuracy on Naive Bayes classifier :  0.803


In [21]:
# SVM classifier 
classifierSVM = BinaryRelevance(SVC())
classifierSVM.fit(X_train_use, y_train)
y_pred_svm= classifierSVM.predict(X_test_use)
print("Accuracy on SVM classifier : ", accuracy_score(y_test, y_pred_svm))

Accuracy on SVM classifier :  0.8987


<b>Which classifier of the three performed better, and why do you think so ?</b>

<b>Answer :</b> 
Average Accuracy for each of following Classifiers is (on 10000 datapoints) :

1. For Naive bayes Classifier - 0.803
2. For SVM Classifier - 0.8987
3. For ANN(implemented from Scratch) - 0.9721
4. For MLPClassifier - 0.9633

Clearly, MLPs Classifier( Both scratch ANN and skleran MLP) outperforms other 2 classifier in terms of accuracies. This is because SVM and Naive Bayes are linear classifier. MLPs are non-linear functions, Hence do not have a linear boundary to separate the two labels.

#Multi-layer Perceptron

For Multi-layer Perceptron, first you will be implementing your own articial Neural network, and then compare it with sklearn's MLP layer.


Implement your own artificial Neural Network (MLP)
Few Steps to be followed(for your reference):

*   Initialise the weights randomly
*   Decide upon the number of layers you wanna have and number of neurons associated in each layer. ( Have a clear idea of what will be the size of the weight matrices).
*   Write the code for forward and backward propogations
*   Decide upon the loss function
*   Train the model
*   Predict the labels after training the model




In [22]:
# you can remove returns statements if u think they are not required
# you can also add functions if needed

def loss(y_pred, y_true):
    J = -1 * (y_true*np.log(y_pred) + (1- y_true)*np.log(1-y_pred))
    return J

def loss_derivative(y_pred, y_true):
    return ((1-y_true)/(1-y_pred)) - (y_true/y_pred)
    
def sig_derivative(z):
    temp = np.ones((z.shape[0],1))
    temp = np.subtract(temp,z)
    return np.multiply(z,temp)

def sigmoid(x):
    return (1/(1+np.exp(-1*x)))
    
def initialise_weights(hidden_layers_size, input_size):
    W = []
    b = []
    no_of_hidden_layers = len(hidden_layers_size)
    for i in range(no_of_hidden_layers):
        if i==0:
            w_temp = np.random.rand(input_size, hidden_layers_size[i])
        else:
            w_temp = np.random.rand(hidden_layers_size[i-1], hidden_layers_size[i])
            b_temp = np.random.rand(hidden_layers_size[i],1)
            b.append(2*b_temp - 1.0)
        W.append(2*w_temp - 1.0)
        
    w_temp = np.random.rand(hidden_layers_size[no_of_hidden_layers-1], 1)
    W.append(2*w_temp-1.0)
    
    b_temp = np.random.rand(1,1)
    b.append(2*b_temp-1.0)

    W = np.array(W)
    b = np.array(b)
    return W, b

def forward_layer(W, b, x):
    A = []
    Z = []
    for i in range(W.shape[0]):
        if i==0:
            a_temp = np.matmul(W[i].T,x)
            z_temp = sigmoid(a_temp)
        else:
            a_temp = np.add(np.matmul(W[i].T, Z[i-1]),b[i-1])
            z_temp = sigmoid(a_temp)        
        A.append(a_temp)
        Z.append(z_temp)
        
    return A,Z


def update_weights(W, b, dW, db, lr):
  l = len(W)
  for i in range(l):
    W[i] = W[i] - lr * dW[l-i-1]
  
  l1 = len(b)
  for i in range(l1):
    b[i] = b[i] - lr * db[l1-i-1]

  return W,b


def backward_propogation(W, Z, y_pred, y_true,x,b):
    
    loss_d = loss_derivative(y_pred, y_true)
    l = W.shape[0]
    dW = []
    db = []

    history = loss_d * sig_derivative(Z[l-1])

    for i in range(W.shape[0]-1):
      temp_dW = np.dot(Z[l-i-2], history.T)
      temp_db = history

      dW.append(temp_dW)
      db.append(temp_db)

      temp_history = W[l-i-1] * sig_derivative(Z[l-i-2])
      history = np.dot(temp_history, history)
  
    temp_dW = np.dot(x, history.T)
    temp_db = history

    dW.append(temp_dW)
    db.append(temp_db)
      
    return dW, db

In [23]:
def train(hidden_layers_size, input_size, lr, X_train, y_train, epochs):

    W_final = 0
    b_final = 0
    min_loss = 1000
    W, b = initialise_weights(hidden_layers_size, input_size)
    for j in range(epochs):
        
        print("Epochs  : ", j+1)
        
        for i in range(len(X_train)):
            temp = X_train[i].reshape(512,1)
            A, Z = forward_layer(W, b, temp)
            y_pred = Z[-1]
            y_true = y_train[i]
            dW,db = backward_propogation(W, Z, y_pred, y_true,temp,b)
            W,b = update_weights(W,b,dW,db,lr)
        
        los = loss(y_pred, y_true)
        
        print("Loss in epoch {} is {}".format(j+1,los))
        
        if los<min_loss:
          min_loss = los
          W_final = W
          b_final = b
    
    return W_final,b_final, min_loss
  

In [24]:
#report accuracies
#here, you need to predict for each of the label as we have trained classifiers in such a way

def predict(W,b, X_test):

    y_pred = []
    for i in range(X_test.shape[0]):
      temp = X_test[i].reshape(512,1)
      A, Z = forward_layer(W, b, temp)
      y_pred.append(np.round(Z[-1][0]))

    return y_pred

In [25]:
hidden_layers_size = [32,4]
input_size = 512
lr = 0.1
epochs = 10
W_list = []
b_list = []
min_loss_list = []
col_list = ['toxic','severe_toxic','obscene', 'threat','insult', 'identity_hate']
for i in range(y_train.shape[1]):
  W,b,min_loss = train(hidden_layers_size, input_size, lr, X_train_use, y_train[col_list[i]],epochs)
  print()
  W_list.append(W)
  b_list.append(b)
  min_loss_list.append(min_loss)

Epochs  :  1
Loss in epoch 1 is [[0.01342349]]
Epochs  :  2
Loss in epoch 2 is [[0.00883221]]
Epochs  :  3
Loss in epoch 3 is [[0.00864352]]
Epochs  :  4
Loss in epoch 4 is [[0.00930528]]
Epochs  :  5
Loss in epoch 5 is [[0.01014396]]
Epochs  :  6
Loss in epoch 6 is [[0.01055624]]
Epochs  :  7
Loss in epoch 7 is [[0.01040868]]
Epochs  :  8
Loss in epoch 8 is [[0.00995689]]
Epochs  :  9
Loss in epoch 9 is [[0.00949283]]
Epochs  :  10
Loss in epoch 10 is [[0.00918189]]

Epochs  :  1
Loss in epoch 1 is [[0.01034893]]
Epochs  :  2
Loss in epoch 2 is [[0.00056635]]
Epochs  :  3
Loss in epoch 3 is [[0.00018649]]
Epochs  :  4
Loss in epoch 4 is [[0.00011179]]
Epochs  :  5
Loss in epoch 5 is [[8.17443175e-05]]
Epochs  :  6
Loss in epoch 6 is [[6.76872406e-05]]
Epochs  :  7
Loss in epoch 7 is [[6.03448617e-05]]
Epochs  :  8
Loss in epoch 8 is [[5.56831348e-05]]
Epochs  :  9
Loss in epoch 9 is [[5.1830765e-05]]
Epochs  :  10
Loss in epoch 10 is [[4.78807008e-05]]

Epochs  :  1
Loss in epoch 1 is

In [26]:
#report accuracies on the test dataset 
def evaluation_matrix(y_test, y_pred):
  acc = accuracy_score(y_pred, y_test)
  print("Accurcy Score : ", acc)
  print("Confusion Matix : ")
  cv = confusion_matrix(y_pred, y_test)
  print(cv)
  return acc

In [27]:
acc_list = []
for i in range(len(W_list)):
  y_pred = predict(W_list[i], b_list[i], X_test_use)
  # print(y_pred)
  acc = evaluation_matrix(y_test[col_list[i]], y_pred)
  acc_list.append(acc)
  print()

Accurcy Score :  0.9222
Confusion Matix : 
[[8560  281]
 [ 497  662]]

Accurcy Score :  0.9944
Confusion Matix : 
[[9944   56]
 [   0    0]]

Accurcy Score :  0.9561
Confusion Matix : 
[[9178  182]
 [ 257  383]]

Accurcy Score :  0.9965
Confusion Matix : 
[[9948   20]
 [  15   17]]

Accurcy Score :  0.9626
Confusion Matix : 
[[9355  240]
 [ 134  271]]

Accurcy Score :  0.9895
Confusion Matix : 
[[9852   79]
 [  26   43]]



In [28]:
print("Average Accuracy score for MLP : ", np.mean(acc_list))

Average Accuracy score for MLP :  0.9702166666666668


Now, compare your implemented MLP with the sklearn MLP layer.

In [29]:
#report accuracies both on train dataset and test dataset
acc_list_sklearn_mlp = []
for i in range(6):
  classifier = MLPClassifier(hidden_layer_sizes=(32,4),activation = 'logistic')
  classifier.fit(X_train_use, y_train[col_list[i]])
  y_pred = classifier.predict(X_test_use)
  acc = evaluation_matrix(y_test[col_list[i]], y_pred)
  acc_list_sklearn_mlp.append(acc)
  print()

Accurcy Score :  0.9221
Confusion Matix : 
[[8605  327]
 [ 452  616]]

Accurcy Score :  0.994
Confusion Matix : 
[[9938   54]
 [   6    2]]

Accurcy Score :  0.9585
Confusion Matix : 
[[9242  222]
 [ 193  343]]

Accurcy Score :  0.9954
Confusion Matix : 
[[9942   25]
 [  21   12]]

Accurcy Score :  0.9571
Confusion Matix : 
[[9278  218]
 [ 211  293]]

Accurcy Score :  0.9889
Confusion Matix : 
[[9832   65]
 [  46   57]]



In [30]:
print("Average Accuracy score for sklearn MLP : ", np.mean(acc_list_sklearn_mlp))

Average Accuracy score for sklearn MLP :  0.9693333333333333


<b>Which MLP performed better? 
Understand and Analyse.</b>

<b>Answer -</b> 

- Average Accuracy for scratch MLP - 0.9721
- Average Accuracy for sklearn MLP - 0.9633

If we only see average accuracies, the scratch ANN/MLP performs slightly better than the sklearn MLP. 